# Prophet Forecasting Model
This notebook implements a forecasting model using the Prophet library. It includes data loading, preprocessing, model training, and forecasting for multiple SKUs, along with additional regressors.

In [32]:
# Install necessary libraries
!pip install prophet --quiet
!pip install pandas
!pip install matplotlib

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [33]:
import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt

## Loading and Preparing the Dataset
Load the dataset and perform necessary preprocessing.

In [34]:
# Load dataset
df = pd.read_csv('dataset_pcs.csv')

# Convert 'period' to datetime
df['date'] = pd.to_datetime(df['date'])

# Select only required columns
df = df[['date', 'sku_id', 'qty_total', 'avg_discount_perc_by_goods', 'oos__by_goods', 'sin_month', 'cos_month']]

In [35]:
# Prepare results container
forecast_all = []

# Loop through each unique SKU
sku_list = df['sku_id'].unique()

for sku in sku_list:
    sku_df = df[df['sku_id'] == sku].copy()

    # Skip SKUs with too little data
    if len(sku_df) < 10:
        continue

    # Rename columns to match Prophet requirements
    sku_df.rename(columns={'date': 'ds', 'qty_total': 'y'}, inplace=True)

    # Drop rows with missing values in regressors
    sku_df = sku_df.dropna(subset=[
        'avg_discount_perc_by_goods',
        'oos__by_goods',
        'sin_month',
        'cos_month'
    ])

    # Skip if less than 2 rows after cleaning
    if len(sku_df) < 2:
        continue

    # Initialize Prophet with additional regressors
    model = Prophet()
    model.add_regressor('avg_discount_perc_by_goods')
    model.add_regressor('oos__by_goods')
    model.add_regressor('sin_month')
    model.add_regressor('cos_month')

    model.fit(sku_df)

    # Create future dataframe for 28 days
    future = model.make_future_dataframe(periods=28)

    # Add future regressor values (using last known value)
    for col in ['avg_discount_perc_by_goods', 'oos__by_goods', 'sin_month', 'cos_month']:
        last_val = sku_df[col].iloc[-1]
        future[col] = last_val  # repeat last known value

    # Forecast
    forecast = model.predict(future)

    result = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(28).copy()
    result.rename(columns={
        'ds': 'date',
        'yhat': 'forecast_qty_total',
        'yhat_lower': 'forecast_qty_total_lower',
        'yhat_upper': 'forecast_qty_total_upper'
    }, inplace=True)

    result['sku_id'] = sku

    forecast_all.append(result)

15:06:18 - cmdstanpy - INFO - Chain [1] start processing
15:06:18 - cmdstanpy - INFO - Chain [1] done processing
15:06:18 - cmdstanpy - INFO - Chain [1] start processing
15:06:20 - cmdstanpy - INFO - Chain [1] done processing


## Saving the Results
Combine all forecasts and save to a CSV file.

In [36]:
# Combine all forecasts
final_forecast_df = pd.concat(forecast_all, ignore_index=True)

# Save to CSV
final_forecast_df.to_csv('forecast.csv', index=False)

# Show sample of result
final_forecast_df.head()

,date,forecast_qty_total,forecast_qty_total_lower,forecast_qty_total_upper,sku_id
0,2023-01-11,197.780411,196.196466,199.316024,SKU001
1,2023-01-12,205.349526,200.352279,210.392698,SKU001
2,2023-01-13,212.918640,203.009139,222.864455,SKU001
3,2023-01-14,220.487754,205.367460,235.721319,SKU001
4,2023-01-15,228.056869,206.733312,249.044645,SKU001
